In [6]:
from multiprocessing.pool import ThreadPool
from pyrodigal import GeneFinder
#from needletail import parse_fastx_file
import Bio.SeqIO
#import click
import sys

In [2]:
def find_genes(seq):
    return (seq.id, GeneFinder(meta=True).find_genes(bytes(seq.seq)))

In [3]:
def translate_genes(input_seqs, f_out):
    with (
            ThreadPool() as pool,
            open(f_out, "w") as f_out
        ):
        for seq_id, pred_genes in pool.imap(find_genes, input_seqs):
            for gene_id, gene in enumerate(pred_genes):
                f_out.write(f">{seq_id}_{gene_id}" + "\n")
                f_out.write(gene.translate() + "\n")

In [9]:
#if __name__ == "__main__":
#    f_in = sys.argv[1]
#    f_out = sys.argv[2]
#    input_seqs = Bio.SeqIO.parse(f_in, "fasta")
#    translate_genes(input_seqs, f_out)

In [4]:
@click.command()
@click.argument("f_in", type=click.File("rb"))
@click.argument("f_out", type=click.File("wb"))
def cli(f_in, f_out):
    input_seqs = Bio.SeqIO.parse(f_in, "fasta")
    translate_genes(input_seqs, f_out)